In [1]:
!pip install comet_ml
# import comet_ml at the top of your file
from comet_ml import Experiment

# Create an experiment with your api key:
experiment = Experiment(
    api_key="1H4oWzrtQdGj3uKZIDjQomMQs",
    project_name="cnn-practice",
    workspace="yusuketakahashi2001",
)

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/yusuketakahashi2001/cnn-practice/1e3d85e8ae304781ba0d0672b71f4619



In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models,transforms,datasets
import torch.nn.functional as F

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [29]:
class CNNNet(nn.Module):
    def __init__(self,in_channel):
        super(CNNNet,self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=in_channel,out_channels=16,kernel_size=3,stride=1),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=1),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1),
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=1),
            # out torch.Size([10, 128, 20, 20])
        )
        self.dence = nn.Sequential(
            nn.Linear(128*20*20,128),
            nn.Dropout(p=0.2),
            nn.Linear(128,64),
            nn.Dropout(p=0.2),
            nn.Linear(64,10),  
        )
    def forward(self,x):
        x = F.relu(self.layer1(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.layer2(x))
        x = F.max_pool2d(x,2,2)
        x = x.view(x.size(0),-1)
        x = self.dence(x)
        
        return x
        
    def check_net_size(self,x):
        x = self.layer1(x)
        out = self.layer2(x)
        
        return out

In [33]:
hyper_params = {
    "sequence_length": 28,
    "num_layers": 2,
    "num_classes": 10,
    "batch_size": 100,
    "num_epochs": 4,
    "learning_rate": 0.01,
    "in_channel":1
   
}

In [35]:
class ImageTransform():
    def __init__(self):
        self.data_out = transforms.Compose([
            transforms.ToTensor()
        ])
        
    def __call__(self,img):
        return self.data_out(img)

In [36]:
def main():
    net = CNNNet(in_channel=hyper_params["in_channel"])
    net.to(device)
    optimizer = optim.Adam(net.parameters(),lr=hyper_params["learning_rate"])
    criterion = nn.CrossEntropyLoss()
    trans = ImageTransform()
    
    #dataload
    dtrain = torch.utils.data.DataLoader(
        datasets.FashionMNIST('./data/fashion-mnist',train=True,download=True,transform=trans),
        batch_size = hyper_params["batch_size"],
        shuffle = True
    )
    
    dtest = torch.utils.data.DataLoader(
        datasets.FashionMNIST('./data/fashion-mnist',train=False,download=True,transform=trans),
        batch_size = hyper_params["batch_size"],
        shuffle = False
    )
    
    loss_train = []
    loss_test = []
    acc_test = []
    
    for epoch in range(1,hyper_params["num_epochs"]+1):
        batch_loss_train = []
        batch_loss_test = []
        correct_total = 0
        total = 0
        
        #train
        net.train()
        for xbatch_train,tbatch_train in dtrain:
            xbatch_train = xbatch_train.to(device)
            tbatch_train = tbatch_train.to(device)
            #Initialize grad
            net.zero_grad()
            ybatch_train = model.forward(xbatch_train)
            lbatch_train = criterion(ybatch_train,tbatch_train)
            lbatch_train.backward()
            optimizer.step()
            batch_loss_train.append(lbatch_train.tolist())
            
        net.eval()
        for xbatch_test,tbatch_test in dtest:
            xbatch_test = xbatch_test.to(device)
            tbatch_test = tbatch_test.to(device)
            ybatch_test = net.forward(xbatch_test)
            lbatch_test = criterion(ybatch_test,tbatch_test)
            batch_loss_test.append(lbatch_test.tolist())
            
            tbatch_label = ybatch_test.argmax(1).to('cpu').numpy()
            tbatch_test = tbatch_test.to('cpu').numpy()
            correct += np.sum((label_test_batch - t_test_batch) == 0)
            total +=1
            
        loss_train.append(np.mean(loss_batch_list_train))
        loss_test.append(np.mean(loss_batch_list_test))
        acc_test = 100*correct/total
        acc_test.append(acc_test_batch)
        
        print('EPOCH: {},TRAIN LOSS:{:.3f},TEST LOSS:{:.3f},TEST ACC:{:.3f}'.format(
            epoch,
            np.mean(loss_list_train),
            np.mean(loss_list_test),
            acc_test_batch
        ))
        
            
        
            
        
        
    
    
    

In [ ]:
if __name__ == '__main__':
    main()